In [ ]:
!pip install -q -U accelerate==0.27.1
!pip install -q -U transformers==4.38.0

In [ ]:
!pip install -q -U datasets==2.16.1

In [1]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay

2024-03-23 17:15:12.372234: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 17:15:12.372300: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 17:15:12.373761: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
checkpoint = "Mr-Vicky-01/English-Tamil-Translator"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
# def language_translator(text):
#     tokenizer = AutoTokenizer.from_pretrained(checkpoint)
#     model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
#     # model = AutoModelForSeq2SeqLM.from_pretrained("finetune-EN-to-Ta/")
#     tokenized = tokenizer([text], return_tensors='pt')
#     out = model.generate(**tokenized, max_length=128)
#     return tokenizer.decode(out[0],skip_special_tokens=True)

In [4]:
raw_datasets = load_dataset("HariprasathSB/tamil_summarization")

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['Summary', 'Text'],
        num_rows: 116936
    })
    test: Dataset({
        features: ['Summary', 'Text'],
        num_rows: 31779
    })
    val: Dataset({
        features: ['Summary', 'Text'],
        num_rows: 25523
    })
})

In [6]:
raw_datasets['train'][0]

{'Summary': 'முக்கியமான நான்கு நவீன கண்டுபிடிப்புக ளை  தாங்களே செய்ததாக சீனா உரிமை கோரும் நிலையில், அத்தகைய உரிமை கோரல்களையும், அது தொடர்பான உண்மைகளையும் ஒப்பிடும் ஓர் அலசல்  இது .',
 'Text': 'உரிமை கோருதல்: அதிவேக ரயில், செல்பேசி வழியாக பணம் செலுத்துதல், இ-வணிகம் மற்றும் மிதிவண்டி பகிர்வு ஆகியவற்றை சீனா கண்டுபிடித்தது. உண்மை சோதனை தீர்ப்பு: இந்த தொழில்நுட்பங்களில் எதையும் சீனா கண்டுபிடிக்கவில்லை. இந்த தொழில்நுட்பங்களை மிகப் பெரிய அளவில் நடைமுறைப்படுத்தும் பாதைக்கு சீனா அழைத்து சென்றுள்ளது. அதிவேக ரயில், செல்பேசி வழியாக பணம் செலுத்துதல், இ-வணிகம் மற்றும் மிதிவண்டி பகிர்வு ஆகியவற்றை சீனா கண்டுபிடித்ததாக உரிமை கோரும் தகவல்கள் 2017ம் ஆண்டு மே மாதம் முதல் சீனாவின் அரசு ஊடகங்களில் மீண்டும் மீண்டும் வெளிவந்தன. சீன தேசிய மக்கள் பேரவையின் பிரதிநிதிகளில் ஒருவரும், சீன இணைய ஜாம்பவான் நிறுவனமான \'டென்சென்று\' வின் முதன்மை செயலதிகாரியாகவும் அறியப்படும் போனி மாவால், சீனாவின் தேசிய மக்கள் பேரவை கூட்டத்தில் சமீபத்தில் இதனை மீண்டும் தெரிவித்துள்ளார். \'த ஹூருன் குளோபல்\' பணக்காரர்கள் பட்டியலின்படி, சீ

In [7]:
max_input_length = 1024
max_target_length = 128
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + str(ex) for ex in examples['Text']]
    targets = [str(ex) for ex in examples['Summary']]
    model_inputs = tokenizer(inputs, max_length=max_input_length,padding="max_length", truncation=True)

    # Setup the tokenizer for targets
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length,padding="max_length",truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
preprocess_function(raw_datasets["train"][1:2])

{'input_ids': [[128022, 4088, 2516, 5598, 9, 11162, 15238, 5167, 74734, 3756, 3000, 23198, 12037, 11376, 7128, 65169, 7422, 42996, 301, 3000, 15707, 5996, 35019, 5283, 2764, 49239, 3452, 7422, 70100, 17190, 17066, 4840, 5996, 16044, 76616, 35768, 2679, 64942, 5, 4167, 20405, 24588, 3690, 18182, 7931, 3000, 15238, 83532, 13488, 13248, 6823, 5, 4167, 13378, 42586, 11968, 19860, 3690, 5654, 3441, 74642, 10230, 60109, 2895, 3585, 88327, 27116, 5538, 54028, 20405, 110280, 5, 40695, 11030, 19985, 94355, 47406, 17900, 19973, 43338, 24168, 5422, 13496, 85850, 41524, 104334, 6087, 20400, 72969, 2679, 71010, 52088, 20730, 2764, 36276, 19973, 20405, 44088, 20422, 5, 99366, 107470, 60317, 67369, 3452, 5900, 21872, 99366, 91395, 6285, 82334, 12416, 42579, 3263, 7931, 4840, 5538, 71175, 3690, 18182, 7931, 3000, 15238, 9810, 104334, 6087, 20400, 56074, 9110, 3316, 19363, 2124, 5392, 51008, 52863, 5, 42447, 9810, 76616, 35768, 7422, 42996, 47840, 10508, 3585, 6538, 3263, 34166, 41132, 9810, 3690, 5654

In [9]:
tokenized_test = raw_datasets.map(preprocess_function, batched=True)

In [10]:
tokenized_test

DatasetDict({
    train: Dataset({
        features: ['Summary', 'Text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 116936
    })
    test: Dataset({
        features: ['Summary', 'Text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 31779
    })
    val: Dataset({
        features: ['Summary', 'Text', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 25523
    })
})

In [11]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model_args = Seq2SeqTrainingArguments(
    output_dir="finetuned-Tamil_summary",  
    overwrite_output_dir=True,  
    do_train=True,  
    logging_dir="logs",  
    logging_steps=50,  # Increase logging frequency for more insights
    save_steps=5000000,  # Save model checkpoints more frequently
    save_total_limit=1,  # Keep more checkpoints
    num_train_epochs=1,  # Extend training to allow for more optimization
    per_device_train_batch_size=2,  # Increase batch size for more efficient training
    gradient_accumulation_steps=2,  # Increase gradient accumulation for stable training
    learning_rate=4e-5,  # Slightly lower learning rate to fine-tune more gently
    warmup_steps=2500,  # Increase warmup steps for better adaptation
    weight_decay=0.01,  
    adam_beta1=0.9,  
    adam_beta2=0.98,  
    adam_epsilon=1e-8,  
    lr_scheduler_type="linear",  
    predict_with_generate=True,  
    fp16=True,  
    seed=42,  
    report_to="wandb",  
    run_name="Tamil_summary",  
)

In [12]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [13]:
trainer = Seq2SeqTrainer(
    model,
    model_args,
    train_dataset=tokenized_test["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [14]:
trainer.train()

wandb: Currently logged in as: vickyvijay069 (vicky12). Use `wandb login --relogin` to force relogin


Step,Training Loss
50,10.722000
100,8.061500
150,6.054900
200,4.870300
250,4.372100
300,4.274300
350,3.752800
400,3.726800
450,3.066500
500,3.048600


KeyboardInterrupt: 

In [15]:
trainer.model.save_pretrained('Tamil-Summarization-model1')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5}


In [33]:
model1 = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working/Tamil-Summarization-model")

In [34]:
def generate_answer(text):
    tokenized = tokenizer([text], return_tensors='pt')
    out = model1.generate(**tokenized, max_length=128)
    return tokenizer.decode(out[0],skip_special_tokens=True)

In [35]:
# text_to_translate = raw_datasets["test"]["en"][5]
output = generate_answer("he was running faster")
print(output)

அவர் விரைவாக ஓடி வந்தார்.


In [49]:
prefix = "summarize: "
output = generate_answer(prefix + """கோட்பாட்டு மொழியியல் பொதுவாக, ஓரளவுக்குத் தனித்தனியாக ஆராயத்தக்க வகையில் பல்வேறு பிரிவுகளாக வகுக்கப்படுகிறது. கீழ்வரும் பிரிவுகள் இன்று பரவலாக ஏற்றுக்கொள்ளப்படுகின்றன:

ஒலிப்பியல் (phonetics), ஒரு மொழியில் பயன்படுத்தப்படும் வெவ்வேறு ஒலிகள் பற்றிய துறை;
ஒலியியல் (phonology), ஒரு மொழியின் அடிப்படை ஒலிகளின் வடிவுரு பற்றி ஆராயும் துறை;
உருபனியல், சொற்களின் உள் அமைப்புப் பற்றி ஆராயும் துறை;
சொற்றொடரியல் (syntax), எவ்வாறு சொற்கள் சேர்ந்து இலக்கணத்துக்குட்பட்ட வசனங்களை உருவாக்குகின்றன என்பதை ஆராயும் துறை;
சொற்பொருளியல் (semantics), சொற்களின் நேரடியான (literal) பொருளை ஆராய்தலுக்கும்,(சொல் குறிக்கும் பொருள் (lexical semantics)), அவை சேர்ந்து உருவாக்கும் வசனங்களின் நேரடியான பொருள்களையும் ஆராயும் துறை;
மொழிநடை (stylistics), மொழியின் பாணிகளை ஆராயும் துறை;
சூழ்பொருளியல் (pragmatics), தொடர்புச் செயற்பாடுகளில் எவ்வாறு utterances பயன்படுத்தப்படுகின்றன (literally, figuratively, அல்லது வேறுவிதமாக) என்பது பற்றிய ஆய்வு;
தொடரியல் (syntax) ஒலி, உருபன்,
இந்த ஒவ்வொரு பகுதியினதும் தனிப்பட்ட முக்கியத்துவம் எல்லோராலும் ஏற்றுக்கொள்ளப்படுவதில்லை, எனினும், கிட்டத்தட்ட எல்லா மொழியியலாளருமே இந்தப் பிரிவுகள் குறிப்பிடத்தக்க அளவு பொதுப் பகுதிகளைக் கொண்டிருக்கின்றன என்பதை ஒத்துக்கொள்வர். இருந்தாலும் ஒவ்வொரு துணைப்பிரிவும், குறிப்பிடத்தக்க அறிவுபூர்வ ஆய்வுகளைச் செய்யக்கூடிய அளவுக்குத் தனியான அடிப்படையான எண்ணக்கருக்களைக் கொண்டுள்ளன.""")
print(output)

கோட்பாட்டு மொழியியல் பொதுவாக, ஓரளவுக்குத் தனித்தனியாக ஆராயத்தக்க வகையில் பல்வேறு பிரிவுகளாக வகுக்கப்படுகிறது. கீழ்வரும் பிரிவுகள் இன்று பரவலாக ஏற்றுக்கொள்ளப்படுகின்றன: ஒலிப்பியல் (phonetics), ஒரு மொழியில் பயன்படுத்தப்படும் வெவ்வேறு ஒலிகள் பற்றிய துறை; ஒலியியல் (phonology), ஒரு மொழியின் அடிப்படை ஒலிகளின் வடிவத்தை பற்றி ஆராயும் துறை; உருபனியல், சொற்களின் உள் அமைப்பு


In [43]:
raw_datasets["test"]["Summary"][13]

'போதை மருந்து வர்த்தகத்தில் ஈடுபட்டதாக கைதாகி சிறையில் இருந்த பிலிப்பைன்ஸ் மேயர் ஒருவர், அவருடைய சிறை அறையிலேயே சுட்டுக் கொல்லப்பட்டுள்ளார்.'

In [31]:
raw_datasets["test"]["Text"][0]

'இது குறித்து அவர் பிபிசி தமிழிடம் கூறுகையில், "இத்தீர்ப்பை மிகச் சிறந்த முற்போக்கான தீர்ப்பாக பார்க்கிறேன். அடிப்படை உரிமை என்ன என்பதை மிகவும் தீவிரமாக இத்தீர்ப்பு விளக்கியுள்ளது" என்றார். "இந்திய அரசியலமைப்பின் 21-ஆவது விதியை மிகவும் ஆழமாக நீதிமன்றம் விளக்கியுள்ளது என்றும், ஏற்கனவே இரு வேறு வழக்குகளில் தனி நபர் அந்தரங்கத்தை அடிப்படை உரிமை பாதுகாக்காது எனக் குறிப்பிட்ட தீர்ப்புகளைத் திருத்தி அந்த உரிமையை தற்போது உச்ச நீதிமன்றம் பாதுகாத்துள்ளது" என்று என்.ராம் கூறினார். "ஆதார் பதிவு விவகாரத்தில் இந்த தீர்ப்பு நிச்சயமாக பிரதிபலிக்கும் என்று கூறும் அவர், ஆதார் முறையைத் திணிக்க முயற்சிக்கும் மத்திய அரசின் எண்ணம் இனி கடினமாக இருக்கும்" என்றார். "நெருக்கடி காலத்தில் நீதிபதி எச்.ஆர். கன்னா அளித்த தீர்ப்பு ஏற்படுத்திய மாற்றத்தைப் போல இந்தத் தீர்ப்பும் சமூகத்தில் மாற்றத்தை ஏற்படுத்தலாம் என்று சிலர் கருதுவதாகவும், மொத்தத்தில் இது ஒரு முக்கியத்துவம் நிறைந்த தீர்ப்பாகும்" என்றும் என்.ராம் தெரிவித்தார். பிற செய்திகள் : சமூக ஊடகங்களில் பிபிசி தமிழ் :'

In [44]:
tokenizer.save_pretrained("model_tokenizer")

('model_tokenizer/tokenizer_config.json',
 'model_tokenizer/special_tokens_map.json',
 'model_tokenizer/vocab.json',
 'model_tokenizer/sentencepiece.bpe.model',
 'model_tokenizer/added_tokens.json')